In [ ]:
#Importing required libraries

import pandas as pd
import deepmatcher as dm
from sklearn.model_selection import train_test_split

In [ ]:
# Importing required dataset

input_data=pd.read_csv('/mnt/c/Users/Exeliq Analytics/Desktop/Deepmatcher/2019-03-26_2019-04-09_suggested_matches_a.csv')

In [ ]:
#Filtering digital source title

dig_data = input_data[input_data['u_original_source_name']=='Digital']

In [ ]:
#Filtering tv_episode title

dig_tv =dig_data [dig_data['u_entity_subtype_name']=='tv_episode']

In [ ]:
#Filtering CEM Manually matched titles

auto_f= dig_tv[dig_tv['is_automatch']=="f"]

In [ ]:
#Exporting to csv for analysis
auto_f.to_csv("/mnt/c/Users/Exeliq Analytics/Desktop/Deepmatcher_r/2019-03-26_2019-04-09_suggested_matches_auto_f.csv")

In [ ]:
#Importing csv 

df= pd.read_csv('/mnt/c/Users/Exeliq Analytics/Desktop/Deepmatcher_r/2019-03-26_2019-04-09_suggested_matches_auto_f.csv')

In [ ]:
#Selecting required columns

data_f=df[["u_entity_id",
"u_name",
"u_episode_number",
"u_season_number",
"u_proto_name",
"m_entity_id",
"m_name",
"m_episode_number",
"m_season_number",     
"m_proto_name",
"status"
]]

In [ ]:
#Renaming columns as per deepmatcher's criteria

f_autodata = data_f.rename(columns={'u_entity_id': 'id', 'u_name': 'left_name',"u_episode_number":"left_episode_number",
                           "u_season_number": "left_season_number","u_proto_name":"left_proto_name",
                            "m_entity_id":"right_id","m_name":"right_name","m_episode_number":"right_episode_number",
                            "m_season_number":"right_season_number",
                            "m_proto_name":"right_proto_name","status":"label"})

In [ ]:
#Creating Key

f_autodata["key"]=f_autodata["id"].astype(str)+" ".join("_")+f_autodata["right_id"].astype(str)

In [ ]:
#Dropping m_entity_id

f_autodata = f_autodata.drop('right_id', 1)

In [ ]:
#Replacing column values

f_autodata.label[f_autodata.label == 'accepted'] = 1 
f_autodata.label[f_autodata.label== 'rejected'] = 0

In [ ]:
#Removing duplicates

uniid_auto_f_data=f_autodata.drop_duplicates(['id'], keep='first')

In [ ]:
#Creating train and test set

train,test = train_test_split(uniid_auto_f_data , test_size=0.35)

In [ ]:
#Creating validation set


train,validation = train_test_split(train , test_size = 0.20)

In [ ]:
#Merging train,test, validation set with the master set

df_train=pd.merge(f_autodata,train["id"],on=["id"],how="inner")
df_test=pd.merge(f_autodata,test["id"],on=["id"],how="inner")
df_validation=pd.merge(f_autodata,validation["id"],on=["id"],how="inner")

In [ ]:
#processing labelled data

train, validation, test = dm.data.process(
    path='/mnt/c/Users/Exeliq Analytics/Desktop/Deepmatcher_r',
    train='train.csv',
    validation='validation.csv',
    ignore_columns='key',
    test='test.csv')

In [ ]:
#defining neural network model

model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
#training model

model.run_train(
    train,
    validation,
    ignore_columns='key',
    epochs=1,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

In [ ]:
#evaluating on test data
results = model.run_eval(test, output_attributes = True)

In [ ]:
#evaluating on unlabelled data

unlabeled = dm.data.process_unlabeled(
    path="/mnt/c/Users/Exeliq Analytics/Desktop/Deepmatcher/testmatches.csv",
    trained_model = model)

In [ ]:
#obtaining predictions

predictions = model.run_prediction(input_data, output_attributes=True)
predictions.head()

In [ ]:
#getting predictions on labelled data

valid_predictions = model.run_prediction(validation, output_attributes=True)
valid_predictions.head()